# Ironhack - Data Analytics Bootcamp
***

## Project 3 - Data gathering processes

Web Scraping and Databases.

### Main Objectives

The main objective of this project is to train your skills on obtaining data from different sources and safely storing it in a database. This will also help your develop skills in designing good process workflows.

<img src="./objectives.jpg" width="600px"/>

### Specific Objectives

1. You will create one code to generate structured data from an API (or API wrapper);
2. You will create another code to generate structured data from raw web-scraping (using beautiful-soup or any related tool you prefer);
3. You will design and create your database with the structured data obtained;
4. Do not forget to document the process.

### Hints

- Keep in mind that you can try to use this database later in the course. So it would be better if you choose something you are familiar with / you are interested in.
- It would be lovely if you could relate both sources. That is, if you could later use both of your sources in the same context.

### Deliverables

- The url of your Jupyter Notebook files on your GitHub.
- The dump of your database.
- [EXTRA] Draw the Entity-Relationship-Diagram of your database (https://app.quickdatabasediagrams.com/#/)

***

## The project:

### Chosing the  sources for this project:

In this project we chose to create a dataframe containg information about the companies that are part of the Standard & Poor's 500, a stock market index that measures the stock performance of 500 large companies listed on stock exchanges in the United States.

After doing some research, we decided to use the Yahoo Finance API¹ which, depite being discontinued,  is free, succinct and contains the quotes of most of the shares contained in S&P 500.

However, this API presented some usage problems. There was a lack of information on some of the tickers, companies and quotes surveyed or the information was out of date.

Therefore, we decided to get most of the information from another source by raw web-scraping. For this 

We did some research and decided to use Wikipedia², considering that this site has a good and reliable table with the information we needed about the S&P 500 companies. In addition, the table proved to be easy to be web-scraped.

Finally we decided that the dataframe to be created would have the following information about the companies:

- Symbol (or ticker);
- Security (or name);
- SEC filings;
- GICS Sector;
- GICS Sub Industry;
- Headquarters Location;
- Date first added;
- CIK;
- Founded;
- 2019 third quarter quotes;
- 2019 fourth quarter quotes;
- 2020 first quarter quotes; and
- 2020 second quarter quotes;

### Sources used:

1 - Yahoo Finance API:
https://rapidapi.com/apidojo/api/yahoo-finance1 or
https://english.api.rakuten.net/apidojo/api/yahoo-finance1

2 - List of S&P 500 companies: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

### Steps taken:

- Importing all the modules and libraries that we needed;
- Saving the url to be scraped in the variable url_sp500;
- Scraping the url using the Bautiful Sopu Python library;
- Getting the first table from the scraped url;
- Creating a pandas dataframe based on the scraped table;
- Saving the tickers from the dataframe into a list to be used to get information from de Yahoo Finance API;
- Searching for data in the Yahoo Finance API using the tickers from the created list;
- Getting the open rates from Yahoo Finance API for the four desired quarters passing the initial dates of the quarters;
- Using parallelization to get the open rates faster;
- Getting the close rates from Yahoo Finance API for the last desired quarter passing the last date of the quarter;
- Using parallelization to get the close rates faster;
- Creating five columns in the sp500 dataframe with the obtained rates;
- Creating a dataframe with all 18 the companies containing NaN to have a register of the companies without useful data;
- Dropping lines in which all values are NaN;
- Cleaning the 'Headquarters Location' column and creating lists of the 'states' and the 'countries':
- Showing the Final dataframe containing 487 of the 505 S&P500 stocks; and
- Exporting the final dataframe to a csv format file.

### Problems faced:

- The Yahoo Finance API appears to be out of date and is slow to work;
- if the imputed date is a weekend day or a holiday, it will not return any value, such as the quote from the previous Friday or the last valid date recorded;
- Yahoo Finance API does not return rates for some dates or some companies; and
- 'Headquarters Location' column had data in different patterns and had to be cleaned. 

### Technologies used:

- Python;
- Pandas;
- Numpy;
- Beautiful Soup;
- Yahoo Finance API;
- Future;
- Json;
- Multiprocess;
- Operator;
- Regex;
- Requests;
- Tqdm; and
- Urllib.

***
## Colaborators:

- Marcus Brandão

- Pelle Adamsen
***

In [ ]:
# !pip install multiprocess

In [ ]:
# All the modules imported:

from bs4 import BeautifulSoup
from __future__ import division
from multiprocess import Pool, cpu_count
from operator import attrgetter
from tqdm import tqdm
import json
import numpy as np
import pandas as pd
import re
import requests
import urllib.request
import yfinance as yf

In [ ]:
# Url to be scrapped to get S&P 500 tickers and some data from Wikipedia:

url_sp500 = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

In [ ]:
# Scrapping the url using "Beautiful Soup":

response_sp500 = requests.get(url_sp500)
html_sp500 = response_sp500.content
soup_sp500 = BeautifulSoup(html_sp500)
soup_sp500.prettify()

In [ ]:
# Getting the first table from the scraped url:
soup_sp500.find_all('table', attrs={'id': 'constituents'})

table = soup_sp500.find_all('table', attrs={'id': 'constituents'})[0]
# table = soup.find('table', attrs={'class':'sortable wikitable'})
table

In [ ]:
# Creating the headers from the new dataframe from the header of the scraped table:

headers = [header.text.strip() for header in table.find_all('th')]

print(headers)

In [ ]:
# Getting a list with all the table data from the scraped table:

data = [data.text.strip() for data in table.find_all('td')]

data

In [ ]:
# Creating a pandas dataframe based on the scraped table:

nrows = int(len(data)/9)
ncols = 9
sp500 = pd.DataFrame(np.array(data).reshape((nrows, ncols)), columns=headers)
sp500

In [ ]:
# Getting a list of the S&P500 tickers from the new table:

sp500_tickers = sp500['Symbol'].values.tolist()

print(sp500_tickers)

In [ ]:
# Searching for data in the Yahoo Finance API using the tickers from the previos created table:

yf_tickers = [yf.Ticker(i) for i in sp500_tickers]

yf_tickers

In [ ]:
# Getting open rates from Yahoo Finance API:

def get_open_rates(i):
    try:
        first_period_open = (i).history(period = '1d', start = '2019-7-1', end = '2019-7-2', rounding = True)['Open'].values.tolist()[0]
    except:
        first_period_open = None
    try:
        second_period_open = (i).history(period = '1d', start = '2019-10-1', end = '2019-10-2', rounding = True)['Open'].values.tolist()[0]
    except:
        second_period_open = None
    try:
        third_period_open = (i).history(period = '1d', start = '2020-1-2', end = '2020-1-3', rounding = True)['Open'].values.tolist()[0]
    except:
        third_period_open = None
    try:
        fourth_period_open = (i).history(period = '1d', start = '2020-4-1', end = '2020-4-2', rounding = True)['Open'].values.tolist()[0]
    except:
        fourth_period_open = None
    return [first_period_open, second_period_open, third_period_open, fourth_period_open]

In [ ]:
# Pool to create a list with all open rates searched in the Yahoo Finance API:

open_rates = []
pool_2 = Pool(processes=cpu_count())
open_rates = pool_2.map(get_open_rates, yf_tickers)

pool_2.terminate()


In [ ]:
# List with all open rates searched in the Yahoo Finance API:

open_rates

In [ ]:
# Getting close rates from Yahoo Finance API:

def get_close_rates(i):
    try:
        fourth_quarter = (i).history(period = '1d', start = '2020-6-29', end = '2020-6-30', rounding = True)['Close'].values.tolist()[0]
    except:
        fourth_quarter = None
    return fourth_quarter

In [ ]:
# Pool to create a list with all close rates searched in the Yahoo Finance API:

close_rates = []
pool_2 = Pool(processes=cpu_count())
close_rates = pool_2.map(get_close_rates, yf_tickers)

pool_2.terminate()


In [ ]:
# List with all close rates searched in the Yahoo Finance API:
close_rates

In [ ]:
print(len(close_rates))

In [ ]:
# Creating five columns in the sp500 dataframe with the obtained rates:

sp500['2019_july_1'] = [i[0] for i in open_rates]
sp500['2019_october_1'] = [i[1] for i in open_rates]
sp500['2020_january_2'] = [i[2] for i in open_rates]
sp500['2020_april_1'] = [i[3] for i in open_rates]
sp500['2020_june_30'] = [i for i in close_rates]


In [ ]:
# Visualizing the sp500 dataframe:

sp500

In [ ]:
# Creating a dataframe with all the rows containing NaN:

sp500_nan = sp500[sp500.isna().any(axis=1)]
sp500_nan

In [ ]:
# Dropping lines in which all values are NaN

sp500 = sp500.dropna(how='any',axis=0)

In [ ]:
sp500.head()
len(sp500)

In [ ]:
# Creating a deep copy of sp500 dataframe to treat the 'Headquarters Location' column:

sp500_copy = sp500.copy()
sp500_copy

In [ ]:
# Cleaning the 'Headquarters Location' column and creating lists of the 'states' and the 'countries':                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           #Cleaning the 'Headquarters Location' column:

us_states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

headquarters = sp500_copy['Headquarters Location'].values.tolist()
location = []
state = []
country = []

for i in headquarters:
    i_splited = i.split(',')

    a = i_splited[-2].strip()
    b = i_splited[-1].strip()
    a = re.sub("\[.\]", "", a)
    b = re.sub("\[.\]", "", b)
    if a == 'VA':
        a = 'Virginia'
    elif b == 'VA':
        b = 'Virginia' 
    if b in us_states:
        location.append(a)
        state.append(b)
        country.append('United States of America')
        print(location, state, country)
    elif a in us_states:
        location.append(b)
        state.append(a)
        country.append('United States of America')
        print(location, state, country)
    else:
        location.append(a)
        state.append(a)
        country.append(b)

for n, i in enumerate(country):
    if i == 'UK':
        country[n] = 'United Kingdom'
    elif i == 'Kingdom of the Netherlands':
        country[n] = 'Netherlands'
    

In [ ]:
# Assigning the lists created above to the three new location columns:

sp500_copy['Headquarters Location'] = location
sp500_copy['State'] = state
sp500_copy['Country'] = country

In [ ]:
# Printing a list of the sp500_copy dataframe columns to reogarnize and rename them:

list(sp500_copy.columns.values)

In [ ]:
# Renaming the columns:

sp500_copy = sp500_copy[['Symbol', 'Security', 'SEC filings', 'GICS Sector', 'GICS Sub Industry', 'Headquarters Location','State', 'Country', 'Date first added', 'CIK', 'Founded', '2019_july_1', '2019_october_1', '2020_january_2', '2020_april_1', '2020_june_30']]

In [ ]:
# Final dataframe containing 498 of the 505 S&P500 stocks: 

sp500_copy

In [ ]:
# Exporting the final dataframe:

sp500_copy.to_csv('./sp500.csv', index=False)